# Operations on fMRI-preprocessing results

- Calculate the input required for LEiDA-net, ELA, and EPLSA

- Conduct brain dynamic analysis using different methods and calculate brain dynamic indicators

- Comparison between groups

- The subsequent application to the natural sleep dataset and the OASIS3 dataset is similar

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
from scipy.stats import ttest_ind,levene,ks_2samp
from scipy.stats import permutation_test
from statsmodels.stats.multitest import multipletests
import math
from scipy.stats import pearsonr
import pickle

- Define the required functions in advance

In [20]:
##计算eigenvector
from scipy.signal import hilbert
from scipy.sparse.linalg import eigs
from nilearn.signal import clean
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr

# Signals utils
def fourier_frec(signal_ts,T):
    sr = 1./T
    if signal_ts.ndim == 1:
        signal_f = np.abs(np.fft.fft(signal_ts))
        f = np.arange(0, sr / 2, 1. * sr / signal_ts.size)
        return f, signal_f[:f.size]
    elif signal_ts.ndim == 2:
        signal_f = np.abs(np.fft.fft(signal_ts,axis=1))
        f = np.arange(0, sr / 2, 1. * sr / signal_ts.shape[1])
        return f, signal_f[:, :f.size]
    else:
        raise Exception('signal_ts has more than 2 dimensions')

def hilbert_phase(signals):

    phase = hilbert(signals, axis=1)
    N_rois = phase.shape[0]
    for roi in range(N_rois): 
        phase[roi, :] = np.angle(phase[roi, :]).real
    phase = phase.real
    return phase

def ang_shortest_diff(a,b):
    if np.abs(a-b)> np.pi:
        c = 2*np.pi-np.abs(a-b)
    else:
        c = np.abs(a-b)
    return c

def clean_signals(signals,detrend=True,standardize='zscore',filter_type=None,low_pass=None,high_pass=None,TR=None):
    if not isinstance(signals,dict):
        raise ValueError("'signals' must be a dictionary!")
    
    cleaned_signals = {}
    
    for sub in signals.keys():
        cleaned_signals[sub] = clean(
            signals[sub].T,
            detrend=detrend, 
            standardize=standardize,
            filter=False if filter_type is None else filter_type, 
            low_pass=low_pass, 
            high_pass=high_pass, 
            t_r=TR
            ).T

    return cleaned_signals


# Matrix utils
def phase_coherence(signals_phases):
    if not isinstance(signals_phases,np.ndarray) or (isinstance(signals_phases,np.ndarray) and signals_phases.ndim!=2):
        raise Exception("'signals_phases' must be a 2D array.")
    
    N = signals_phases.shape[0] #number of voxels/parcels
    T = signals_phases.shape[1]-2 #number of time points/volumes

    dFC = np.zeros((N,N,T)) #matrix to save the phase-coherence between regions n and p at time t.
    signals_phases = signals_phases[:,1:-1] #delete the fist and last time point of the time series of each ROI signal.
    
    for time_point in range(T): #for each time point:
        for roi_1 in range(N): #for each region of interest
            for roi_2 in range(N): #relate with other region of interest
                dFC[roi_1,roi_2,time_point] = np.cos(
                    ang_shortest_diff(
                        signals_phases[roi_1,time_point],
                        signals_phases[roi_2,time_point]
                        )
                    )
    return dFC

def get_eigenvectors(dFC,n=1):
    if not isinstance(dFC,np.ndarray) or (isinstance(dFC,np.ndarray) and dFC.ndim!=3):
        raise Exception("'dFC' must be a 3D array!")
    
    T, N = dFC.shape[-1], dFC.shape[0] #number of time points and number of regions
    
    LEi = np.empty((T,n*N))
    for t in range(T):
        avals, avects = eigs(dFC[:,:,t], n, which='LM')
        ponderation = avals.real / np.sum(avals.real)
        for x in range(avects.shape[1]):
            # convention, negative orientation
            if np.mean(avects[:, x] > 0) > .5:
                avects[:, x] *= -1
            elif np.mean(avects[:, x] > 0) == .5 and np.sum(avects[avects[:, x] > 0, x]) > -1. * sum(avects[avects[:, x] < 0, x]):
                avects[:, x] *= -1

        LEi[t] = np.hstack([p * avects.real[:, x].real for x, p in enumerate(ponderation)])

    return LEi

In [21]:
def load_tseries(path):
    try:
        signals_path = f'{path}/time_series'
        sub_folders = [f for f in os.listdir(signals_path) if os.path.isdir(f'{signals_path}/{f}')]
        sub_folders.sort()
        tseries = {}
        for sub in sub_folders:
            for file in os.listdir(f'{signals_path}/{sub}'):
                if file.endswith('.csv'):
                    tseries[sub] = np.array(
                                        pd.read_csv(
                                            f'{signals_path}/{sub}/{file}',
                                            sep=',',
                                            header=None
                                            )
                                        )
                    tseries[sub] = tseries[sub][1:, :]
            
        return tseries
    except:
        raise Exception("The time series couldn't be loaded.")
def load_classes(data_path):
    try:
        try:
            classes = load_dictionary(f'{data_path}/metadata.pkl')
        except:
            metadata = pd.read_csv(f'{data_path}/metadata.csv',sep=',')
            cols = ['subject_id','condition']
            if not all(item in cols for item in list(metadata.columns)):
                raise Exception("f'{cols} columns must be present in 'metadata.csv'!")
            classes = {}
            for sub in np.unique(metadata.subject_id):
                classes[sub] = [label for label in metadata[metadata.subject_id == sub].condition]

        return classes
    except:
        raise Exception("The groups/conditions labels coudn't be loaded.")
def txt2list(path):
    """
    Load a .txt file as a list.
    Note: the .txt file must contain
    an entry/value per line/row.

    Params:
    -------
    path : str.
        Full path to the .txt file of
        interest. E.g.: 'data/rois_labels.txt'

    Returns:
    --------
    list_ : list.
    """
    with open(path,'r') as file:
        list_ = [line.strip() for line in file]
    return list_

def load_rois_labels(path):
    try:
        return txt2list(f'{path}/rois_labels.txt')
    except: 
        raise Exception("The ROIs' labels couldn't be loaded "
                        "from the provided path.")

In [22]:
from itertools import groupby
import pickle
def dwell_times(labels, TR=None, plot=False):
    
    dwell = {}
    for cluster in np.unique(labels):  # for each cluster (aka brain state)
        dwell[cluster] = []
        dwell[cluster].append([len(list(g[1])) for g in groupby(labels) if g[0]==cluster])  # get a sequence with the times it appears across time.
    
    # transforming list to array
    for cluster in dwell.keys(): 
        dwell[cluster] = np.concatenate([i for i in dwell[cluster]])
    
    # compute the average lifetime (TRs or seconds) of each state    
    if TR is None:
        mean_dwell = pd.DataFrame({'Cluster':[i for i in dwell.keys()],'Mean_lifetime':[np.mean(dwell[i]) for i in dwell.keys()]})
    else:
        mean_dwell = pd.DataFrame({'Cluster':[i for i in dwell.keys()],'Mean_lifetime':[np.mean(dwell[i])*TR for i in dwell.keys()]})
    
    if plot:
        plt.figure()
        plt.grid()
        sns.barplot(x=[f'State {i+1}' for i in range(len(mean_dwell))], y=mean_dwell.Mean_lifetime)
        if TR is None:
            plt.ylabel('Average lifetime (TRs)', fontsize=15, fontweight='regular') 
        else:
            plt.ylabel('Average lifetime (s)', fontsize=15, fontweight='regular')
        plt.xlabel('State', fontsize=15, fontweight='regular') 
        plt.tight_layout()
        plt.show()
                                            
    return dwell, mean_dwell


def dwell_times_group(metadata,labels,TR=None,save_results=False,path=None):
    if save_results and path is None: 
        raise Exception('You must provide a path to save the results')
    assert metadata.shape[0] == labels.shape[0], \
        "The number of rows in 'metadata' must be the same as in 'labels'"
    
    results = []
    #N_clusters = len(np.unique(labels))
    for cond in np.unique(metadata.condition):
        subs = metadata[metadata.condition==cond].subject_id.values
        for s in np.unique(subs):
            idx = np.logical_and(metadata.subject_id==s,metadata.condition==cond)
            dw = dwell_times(labels[idx],TR=TR,plot=False)[-1]
            dw_ = {
                **{'subject_id':s,'condition':cond},
                **{f'PL_state_{k}':v for k,v in zip(dw.Cluster,dw.Mean_lifetime)}
                }
            results.append(dw_)
    
    results = pd.DataFrame(results).fillna(0.0)

    #reorder columns by PL_state (necessary because, when a subject do not traverse
    # a particular state, that state will be located at the end of the dataframe columns)
    N_states = len(results.columns)-2
    states_columns = [f'PL_state_{i+1}' for i in range(N_states)]
    results = results[['subject_id','condition']+states_columns]

    if save_results:
        try: 
            results.to_csv(f'{save_results}/dwell_times.csv',sep='\t',index=False)
        except:
            print("Warning: 'dwell_times.csv' was not saved in local folder.")

    return results

In [23]:
def transition_probabilities(labels,k,norm=True,plot=False):
    """
    Compute the transitions between patterns across time
    for a single subject. The number of states ('k')
    defines the matrix shape, ensuring that, if a given
    subject don't traverse a state of the detected states
    for the whole group, the matrix will be constructed
    correctly, respecting the original number of states.
    If a subject don't traverse a particular state,
    then the corresponding row will contain all zeros.
    
    Params:
    ------
    labels : ndarray with shape (N_time_points,). 
        Contains the KMeans predicted label
        for each time point.

    norm : bool. 
        Whether to normalize the values of the
        transitions matrix.

    plot : bool. 
        Whether to plot the transitions matrix.
    
    Returns:
    --------
    transitions : ndarray with shape (N_states,N_states).
    """

    N_states = k
    transitions = np.zeros((N_states, N_states)) #create empty matrix 
    N_volumes = labels.size 

    #compute the transitions
    for volume_idx in range(N_volumes - 1):
        _from = labels[volume_idx]
        _to = _from + labels[volume_idx+1] - labels[volume_idx]
        transitions[_from-1,_to-1]+=1

    if norm:
        np.seterr(invalid='ignore') # hide potential warning when dividing 0/0 = NaN
        transitions = np.divide(
            transitions.astype(np.float_),
            np.sum(transitions,axis=1).reshape(transitions.shape[1],1)
            ) #normalize the values to probabilities

    #replace nan (if any) with 0. Can happen when a subject
    # doesn't traverse a state at all.
    transitions = np.nan_to_num(transitions,copy=True)
        
    if plot:
        plt.figure()
        sns.heatmap(
            transitions,
            annot=True,
            cmap='viridis', 
            square=True,
            linecolor='black',
            linewidths=0.5,
            xticklabels=[f'State {i+1}' for i in range(N_states)],
            yticklabels=[f'State {i+1}' for i in range(N_states)],
            cbar_kws={"shrink": 0.5}
            )
        plt.yticks(rotation='horizontal')
        plt.xlabel('To',fontsize=15,fontweight='regular')
        plt.ylabel('From',fontsize=15,fontweight='regular')
        plt.tight_layout()

    return transitions

def transition_probabilities_group(metadata,labels,save_results=False,path=None):
    assert metadata.shape[0] == labels.shape[0], \
        "The number of rows in 'metadata' must be the same as in 'labels'"
    if save_results and path is None: 
        raise Exception('You must provide a path to save the results')

    results = []
    subjects_list,cond_list = [],[]
    N_clusters = len(np.unique(labels))
    for cond in np.unique(metadata.condition):
        subs_ids = metadata[metadata.condition==cond].subject_id.values #get an array with the subjects ids that belongs to a given condition.
        for subject in np.unique(subs_ids):
            idx = np.logical_and(metadata.subject_id==subject,metadata.condition==cond)
            tr = transition_probabilities(labels[idx],k=N_clusters,norm=True,plot=False)
            results.append(np.ravel(tr)) #vectorize the transitions matrix.
            subjects_list.append(subject)
            cond_list.append(cond)

    results = pd.DataFrame(np.vstack(results))
    columns_names = []
    for i in range(N_clusters):
        for j in range(N_clusters):
            columns_names.append(f'From_{i+1}_to_{j+1}')
    
    results.columns = columns_names
    results.insert(0,'subject_id',subjects_list)
    results.insert(1,'condition',cond_list)

    if save_results: 
        try:
            results.to_csv(f'{path}/transitions_probabilities.csv',sep='\t',index=False)
        except:
            print("The results could't be saved in .csv file. Please check the provided path.")

    return results

In [ ]:
def hedges_g(x1,x2,paired=False):
    #sample sizes
    n1 = x1.size
    n2 = x2.size
    
    #degrees of freedom
    dof = n1+n2-2
    
    #variances
    var1 = np.var(x1)
    var2 = np.var(x2)
    
    #difference in means
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    diff_mean = np.abs(m1-m2)
    
    #pooled standard deviation
    #s1 = np.std(x1)
    #s2 = np.std(x2)
    
    #Hedges's g
    if not paired:
        s_pooled = np.sqrt(
            (((n1-1)*var1)+((n2-1)*var2))/dof
            )
        g = diff_mean/s_pooled

    else:
        g = diff_mean / np.sqrt((var1+var2) / 2)

    return g

def permtest_ind(data,class_column=None,n_perm=5_000,alternative='two-sided'):
    #Validation of input data
    if class_column is None:
        raise ValueError("You must specify the 'class_column'.")
    elif class_column not in data.columns:
        raise ValueError(f"The 'class_column' '{class_column}' not founded in 'data'!")
    if not isinstance(n_perm,int):
        raise TypeError("'n_perm' must be an integer!")
    
    features = [col for col in data if col!=class_column] #list with variables to be tested
    n_tests = len(features) #number of tests that will be executed
    groups = np.unique(data[class_column]) #get the groups names.
    results = [] #list to save results
    p_values = []
    
    for col in features:
        x1 = data[data[class_column]==groups[0]][col].values #data of first group.
        x2 = data[data[class_column]==groups[1]][col].values #data of the other group.

        #running Levene's test
        _,p_levene = levene(x1,x2,center='mean')

        #running the permutation test
        test = ttest_ind(
            x1,
            x2,
            alternative=alternative,
            permutations=n_perm,
            equal_var=True if p_levene>0.05 else False
        )
#         p_values.append = [test.pvalue]
        p_values.append(test.pvalue)
#         _, p_adjusted, _, _ = multipletests(p_values, method='fdr_bh')
        #computing effect size
        eff = hedges_g(x1,x2)

        results.append({
            #'k': k,
            'variable':col, 
            'group_1':groups[0],
            'group_2':groups[-1],
            'statistic':test.statistic,
            'p-value':test.pvalue,
            'test':'t-test' if p_levene>0.05 else 'welch',
            'effect_size':eff,
#             'p-value_FDR':p_adjusted,
#             'reject_null_FDR':True if p_adjusted < 0.05 else False
            })

    results = pd.DataFrame(results)

    _, p_adjusted, _, _ = multipletests(p_values, method='fdr_bh')
    results['p-value_FDR'] = p_adjusted
    results['reject_null_FDR'] = [True if p < 0.05 else False for p in p_adjusted]

    return results


def _statistic(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

In [ ]:
def get_dynamics(basin_number, nmin):
    # Get Dynamics
    num = len(basin_number)
    
    # Direct transition between basin
    dtrans = np.zeros((nmin, nmin))
    
    for i in range(nmin):
        for j in range(nmin):
            if i == j:
                continue
            cnt = 0
            for t in range(num - 1):
                if basin_number[t] == i+1 and basin_number[t + 1] == j+1:
                    cnt = cnt + 1
#                 cnt = np.sum((basin_number[:-1] == i) & (basin_number[1:] == j))
            dtrans[i, j] = cnt / num
    
    # Direct and indirect transition between basin
    trans = np.zeros((nmin, nmin))

    for i in range(nmin):
        for j in range(nmin):
            if i == j:
                continue
            ind_i = basin_number == i+1
            ind_j = basin_number == j+1
            cnt = 0

            st_i = np.where(ind_i)[0]
            if len(st_i) == 0:
                continue
            if st_i[0] >= num:
                continue

            st_j = np.where(ind_j)[0]
            if len(st_j) == 0:
                continue
            if st_j[0] >= num:
                continue

            st_ii = st_i[0]
            while True:
                tmp_j = np.where(st_j > st_ii)[0]
                if len(tmp_j) == 0:
                    break
            
                cnt += 1
            
                st_jj = st_j[tmp_j[0]]
                tmp_i = np.where(st_i > st_jj)[0]
                if len(tmp_i) == 0:
                    break
                st_ii = st_i[tmp_i[0]]

            trans[i, j] = cnt / num
    
    # Set return value
    dynamics = {
        'Dtrans': dtrans,
        'Trans': trans
    }
    
    return dynamics

In [26]:
##FS
def calculate_nodal_functional_segregation_strength(ts, modu1, modu2, num_parcel):
    node_fs = [None] * num_parcel

    for i in range(num_parcel):
        fc_in = []
        fc_ac = []
        if i in modu1:
            for j in modu1:
                if j != i:
                    corr_coeff, p_value = pearsonr(ts[:, i], ts[:, j])
                    fc_in.append(0.5 * math.log((1 + corr_coeff) / (1 - corr_coeff)))
            for j in modu2:
                corr_coeff, p_value = pearsonr(ts[:, i], ts[:, j])
                fc_ac.append(0.5 * math.log((1 + corr_coeff) / (1 - corr_coeff)))
        if i in modu2:
            for j in modu1:
                corr_coeff, p_value = pearsonr(ts[:, i], ts[:, j])
                fc_ac.append(0.5 * math.log((1 + corr_coeff) / (1 - corr_coeff)))
            for j in modu2:
                if j != i:
                    corr_coeff, p_value = pearsonr(ts[:, i], ts[:, j])
                    fc_in.append(0.5 * math.log((1 + corr_coeff) / (1 - corr_coeff)))

        node_fs[i] = np.mean(fc_in) - np.mean(fc_ac)

    return node_fs

def calculate_functional_segregation_strength(ts, modu1, modu2):
    fc1 = []
    fc2 = []
    fc12 = []
    for i in range(len(modu1)):
        for j in range(i + 1, len(modu1)):
            corr_coeff, p_value = pearsonr(ts[:, modu1[i]], ts[:, modu1[j]])
            fc1.append(0.5 * math.log((1 + corr_coeff) / (1 - corr_coeff)))
    for i in range(len(modu2)):
        for j in range(i + 1, len(modu2)):
            corr_coeff, p_value = pearsonr(ts[:, modu2[i]], ts[:, modu2[j]])
            fc2.append(0.5 * math.log((1 + corr_coeff) / (1 - corr_coeff)))
    for i in range(len(modu1)):
        for j in range(len(modu2)):
            corr_coeff, p_value = pearsonr(ts[:, modu1[i]], ts[:, modu2[j]])
            fc12.append(0.5 * math.log((1 + corr_coeff) / (1 - corr_coeff)))

    within_module_fc = fc1 + fc2
    across_module_fc = fc12

    # Calculate the average within-module FC
    average_within_module_fc = np.mean(within_module_fc)

    # Calculate the average across-module FC
    average_across_module_fc = np.mean(across_module_fc)

    # Calculate the functional segregation strength as the difference between the averages
    functional_segregation_strength = average_within_module_fc - average_across_module_fc

    return average_within_module_fc, average_across_module_fc, functional_segregation_strength


def load_dictionary(filepath):
    with open(filepath, 'rb') as file:
        dict_ = pickle.load(file)
        return dict_

- Based on the Schaefer400 information of the brain atlas, the preprocessed time series is processed according to the network to obtain the network-based time series as the input of the LEiDA-net method

In [ ]:
icn_file = '/data/dzy/bids_INV2/derivatives/fig5-lesion_seed/schaefer400x7_MNI.csv' 
icn_data = pd.read_csv(icn_file)
folder = f'/data/dzy/combined_method/AD/normal_amyloid_group/'
# for i in range(1,185):
i=80
for filename in sorted(os.listdir(folder)):
    folder_path = os.path.join(folder, filename)
    print(folder_path)
    i+=1
    folder_path_net = f'/data/dzy/combined_method/AD/leida_net_amyloid_group/time_series/sub-{i:04d}'
    folder_path_roi = f'/data/dzy/combined_method/AD/leida_roi_amyloid_group/time_series/sub-{i:04d}'
    if not os.path.exists(folder_path_net):
        os.makedirs(folder_path_net)
    if not os.path.exists(folder_path_roi):
        os.makedirs(folder_path_roi)
    bold_data = pd.read_csv(folder_path)
    bold_data = bold_data.T
    bold_data.to_csv(f'/data/dzy/combined_method/AD/leida_roi_amyloid_group/time_series/sub-{i:04d}/sub-{i:04d}.csv',index = False)
    if 'Name' in icn_data.columns:
        bold_data.index = icn_data['Name']
    bold_data = bold_data.T
    new_data = pd.DataFrame()
    for icn in icn_data['ICN'].unique():
        icn_col = icn_data[icn_data['ICN']==icn]['Name'].tolist()
        icn_group = bold_data[icn_col]
        new_data[icn] = icn_group.mean(axis=1)
    new_data = new_data.T
    df = pd.DataFrame(new_data)
    row_means = df.mean(axis=1)
    sign_data = pd.DataFrame(index=df.index, columns=df.columns)
    for col in df.columns:
        for idx in df.index:
            value = df.at[idx, col]
            if value > row_means[idx]:
                sign_data.at[idx, col] = 1
            elif value < row_means[idx]:
                sign_data.at[idx, col] = -1
    new_data.to_csv(f'/data/dzy/combined_method/AD/leida_net_amyloid_group/time_series/sub-{i:04d}/sub-{i:04d}.csv',index = False,sep=',')

- Based on the results of LEiDA-net obtained above, calculate the principal eigenvector and binarize it, which is the input of the EPLSA method. Reference [energy landscape analysis toolbox](https://github.com/tkEzaki/energy-landscape-analysis) classify the brain state and some index calculation

In [ ]:
data_path = '/data/dzy/combined_method/AD/leida_net_amyloid_group'
subject_ids = list(load_tseries(data_path).keys())  # list of subject ids
N_subjects = len(subject_ids)  # number of provided subjects
eigens = []
sub_list, class_list = [], []

# Starting process
print("\n-STARTING THE PROCESS:\n"
         "========================\n"
        f"-Number of subjects: {N_subjects}")

print("\n 1) EXTRACTING THE EIGENVECTORS.\n")
time_series = load_tseries(data_path)
time_series = clean_signals(
    signals=time_series,
    detrend=True, 
    standardize='zscore',  # z-score
    filter_type='butterworth',  #Butterworth
    low_pass=0.1,  
    high_pass=0.01,  
    TR=2.5  
)
classes = load_classes(data_path)
rois_labels = load_rois_labels(data_path)
for sub_idx, sub_id in enumerate(subject_ids):  # for each subject
            # get current subject signals

    tseries = time_series[sub_id]
    N_volumes = tseries.shape[1] - 2

    print(f"SUBJECT ID: {sub_id} ({tseries.shape[1]} volumes)")

    # Extract the eigenvectors from each phase-coherence matrix at time t.
    eigens.append(
        get_eigenvectors(phase_coherence(hilbert_phase(tseries)))
    )

    # Append metadata to lists (to complete the eigenvectors dataset)
    for volume in range(N_volumes):
        sub_list.append(sub_id)
        if len(classes[sub_id]) > 1:
            class_list.append(classes[sub_id][volume + 1])
        else:
            class_list.append(classes[sub_id][0])

df_eigens = pd.DataFrame(np.vstack(eigens), columns=rois_labels)
df_eigens.insert(0, 'subject_id', sub_list)
df_eigens.insert(1, 'condition', class_list)

# saving results
df_eigens.to_csv(f'/data/dzy/combined_method/AD/EPLSA_amyloid_group/eigenvectors.csv', sep='\t', index=False)

In [ ]:
##Obtain the main feature vector data and binarize it
eigenvector_data = pd.read_csv('/data/dzy/combined_method/AD/EPLSA_amyloid_group/eigenvectors.csv',sep='\t')
sub_id = eigenvector_data['subject_id']
unique_sub_IDs = np.unique(sub_id)
print(len(unique_sub_IDs))
for sub_ID in unique_sub_IDs:
    sub_data_subset = eigenvector_data[eigenvector_data['subject_id'] == sub_ID]
    df = pd.DataFrame(sub_data_subset)
    df = df.iloc[:, 2:]
    df = df.T
    sign_data = pd.DataFrame(index=df.index, columns=df.columns)
    for col in df.columns:
        for idx in df.index:
            value = df.at[idx, col]
            if value > 0:
                sign_data.at[idx, col] = 1
            elif value < 0:
                sign_data.at[idx, col] = -1
    sign_data.to_csv(f'/data/dzy/combined_method/AD/EPLSA_amyloid_group/dat/{sub_ID}.dat', index=False, header=False, sep=' ')

825


- Calculate the principal eigenvector based on the results of LEiDA-roi and binarize it, which is the input of the ELA method. Reference [energy landscape analysis toolbox](https://github.com/tkEzaki/energy-landscape-analysis) classify the brain state and some index calculation

In [ ]:
for i in range(1,525):
    data_path = f'/data/dzy/combined_method/ASD_new/200_400/time_series/sub-{i:03d}/sub-{i:03d}.csv'
    sub_data_subset = pd.read_csv(data_path)
    df = pd.DataFrame(sub_data_subset)
    row_means = df.mean(axis=1)
    sign_data = pd.DataFrame(index=df.index, columns=df.columns)
    for col in df.columns:
        for idx in df.index:
            value = df.at[idx, col]
            if value > row_means[idx]:
                sign_data.at[idx, col] = 1
            elif value < row_means[idx]:
                sign_data.at[idx, col] = -1
    sign_data.to_csv(f'/data/dzy/combined_method/ASD_new/200_400/ELA/dat/sub-{i:03d}.dat', index=False, header=False, sep=' ')

- ELA, EPLSA brain dynamic index method, two LEiDA method of index calculating reference [LEiDA](https://github.com/juanitacabral/LEiDA)

In [ ]:
##Calculate the dwell time and transition probability of each subject in each state
labels = []
for i in range(1,292):
    label = pd.read_csv(f'/data/dzy/combined_method/AD/EPLSA_amyloid/result/sub-{i:04d}_BN.csv', header=None)
    label.insert(0, 'subject_id', f'sub-{i:04d}')
    label.insert(1, 'condition', 'AD')
    labels.append(label)
for b in range(292,932):
    label = pd.read_csv(f'/data/dzy/combined_method/AD/EPLSA_amyloid/result/sub-{b:04d}_BN.csv', header=None)
    label.insert(0, 'subject_id', f'sub-{b:04d}')
    label.insert(1, 'condition', 'HC')
    labels.append(label)

labels = pd.concat(labels, ignore_index=True)
meta = labels[['subject_id','condition']]
ys = labels.iloc[:,2].values

dwelltimes = dwell_times_group(
            meta,
            ys,
            TR=2,
            save_results=False,
            path = '/data/dzy/combined_method/AD/EPLSA_amyloid/result'
            )
dwelltimes.to_csv('/data/dzy/combined_method/AD/EPLSA_amyloid/result/dwell_time.csv',sep='\t',index=False)

transitions = transition_probabilities_group(
            meta,
            ys,
            save_results=False,
            path= '/data/dzy/combined_method/AD/EPLSA_amyloid/result'
            )
transitions.to_csv('/data/dzy/combined_method/AD/EPLSA_amyloid/result/transition_probabilities.csv',sep='\t',index=False)

In [ ]:
import datetime

dynamics_results = {}
basin_numbers = labels.iloc[:, 2].values
nmin = 6  
for subject_id in labels['subject_id'].unique():
    subject_basin_numbers = basin_numbers[labels['subject_id'] == subject_id]
    dynamics_results[subject_id] = get_dynamics(subject_basin_numbers, nmin)

# for subject_id, dynamics in dynamics_results.items():
#     print(f"Subject ID: {subject_id}")
#     print("Direct transition between basin:\n", dynamics['Dtrans'])
#     print("Direct and indirect transition between basin:\n", dynamics['Trans'])
#     print()
    
output_file = f'/data/dzy/combined_method/AD/EPLSA_baseline/result_baseall/Transition.csv'  # 构造输出文件名

columns = ['InputFile']
for i in range(nmin):
    for j in range(nmin):
        if i != j:
            columns.append(f'Direct transitions from B{i+1} to B{j+1}')
for i in range(nmin):
    for j in range(nmin):
        if i != j:
            columns.append(f'Transitions from B{i+1} to B{j+1}')

data = []
for k, (subject_id, dynamics) in enumerate(dynamics_results.items()):
    row = [subject_id]
    for i in range(nmin):
        for j in range(nmin):
            if i != j:
                row.append(dynamics['Dtrans'][i, j])
    for i in range(nmin):
        for j in range(nmin):
            if i != j:
                row.append(dynamics['Trans'][i, j])
    data.append(row)

df = pd.DataFrame(data, columns=columns)
df.to_csv(output_file, index=False)

In [ ]:
data_path = '/data/dzy/combined_method/AD/leida_net_baseline'
time_series = load_tseries(data_path)
time_series = clean_signals(
    signals=time_series,
    detrend=True,  
    standardize='zscore',  
    filter_type='butterworth', 
    low_pass=0.1, 
    high_pass=0.01,  
    TR=2.5  
)
modu1 = pd.read_csv('/data/dzy/combined_method/AD/EMSA/6_modu1.csv')
modu1 = modu1['Index'].tolist()
# print(modu1)
modu2 = pd.read_csv('/data/dzy/combined_method/AD/EMSA/6_modu2.csv')
modu2 = modu2['Index'].tolist()
num_parcel = 7
subject_ids = list(time_series.keys())  # list of subject ids
N_subjects = len(subject_ids)  # number of provided subjects
node_fs_list = []
average_within_module_fc_list = []
average_across_module_fc_list = []
functional_segregation_strength_list = []
for sub_idx, sub_id in enumerate(subject_ids):  # for each subject
    # get current subject signals
    tseries = time_series[sub_id]
#     print(tseries)
    tseries = tseries.T
    print(tseries)
    node_fs = calculate_nodal_functional_segregation_strength(tseries, modu1, modu2, num_parcel)
    node_fs_list.append(node_fs)
    average_within_module_fc, average_across_module_fc, functional_segregation_strength = calculate_functional_segregation_strength(tseries, modu1, modu2)
    average_within_module_fc_list.append(average_within_module_fc)
    average_across_module_fc_list.append(average_across_module_fc)
    functional_segregation_strength_list.append(functional_segregation_strength)
results_df = pd.DataFrame({
    'Subject ID': subject_ids,
    'Node FS': node_fs_list,
    'Average_Within_Module_FC': average_within_module_fc_list,
    'Average_Across_Module_FC': average_across_module_fc_list,
    'Functional_Segregation_Strength': functional_segregation_strength_list
})
results_df.to_csv('/data/dzy/combined_method/AD/EPLSA_baseline/result/FS7.csv', index=False)

- Significance analysis

In [ ]:
from itertools import combinations
##Significance analysis of indicators
data = pd.read_csv('/data/dzy/combined_method/AD/EPLSA_amyloid/result_ad39_hc362/Dynamics_20250622_16_28_08_new.csv',sep =',')
data = data.dropna(subset=['dx'])
conditions = np.unique(data.dx)
N_conditions = conditions.size
results_stacked = []
for conds in combinations(conditions,2): 
    data_ = data[data.dx.isin(conds)] 
    stats_results = permtest_ind(
        data_,
        class_column='dx',
        alternative='two-sided',
        n_perm=5_000
        )
    results_stacked.append(stats_results)
if N_conditions>2:
        metric_results = pd.concat(results_stacked,axis=0).reset_index(drop=True)
else:
        metric_results = pd.DataFrame(stats_results)
print(metric_results)
metric_results.to_csv(f'/data/dzy/combined_method/AD/EPLSA_amyloid/result_ad39_hc362/occ_new.csv',sep='\t',index=False)